## UDP Server

In [3]:
import socket
import pickle
import requests

bytes_ErrorAPI = str.encode("Error al generar una pregunta")

host = "127.0.0.1"
port = 6700
server_adress = (host, port)

print("- Waiting for a client...")
print("- Listening in {}:{}".format(host,port))

socket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
socket.bind((host, port))

correctas = 0

#https://opentdb.com/api_config.php
#API Call (Generamos 10 preguntas aleatorias de cualquier tipo)
response = requests.get("https://opentdb.com/api.php?amount=10")

#Continua mandando preguntas hasta que haga todas las que ha enviado la API
for i in range(len(response.json()["results"])):
    
    #Por si falla la API
    if (response.status_code != requests.codes.ok):
        socket.sendto(bytes_ErrorAPI, server_adress)
        print("Error:", response.status_code, response.text)
        break

    bytes_rx = socket.recvfrom(1024)
    packed = bytes_rx[0]
    data = pickle.loads(packed)
    
    server_adress = bytes_rx[1]
    #print("Recieved packed: ", packed)
    
    print("Message from client " + str(data[1]) + ": " + data[0])
    
    #EL cliente pide al server una pregunta
    if (data[0] == "ASK"):
        
        #Response is a JSON
        question = response.json()["results"][i]["question"]

        print("Server sending question: ", question)

        correct_answ = response.json()["results"][i]["correct_answer"]
        incorrect_answrs = response.json()["results"][i]["incorrect_answers"]

        incorrect_answrs.append(correct_answ)
        
        print("All answers: ", incorrect_answrs)

        pack = (question, incorrect_answrs)
        send = pickle.dumps(pack)
        socket.sendto(send, server_adress)
            
    #Comprobación de la respuesta del cliente
    elif (data[0] == "ANSWER"):
        print("Client answer ", data[2])

        if (data[2] == correct_answ):
            print("Client sent correct answer.")
            socket.sendto(pickle.dumps(("Correcto!")), server_adress)
            correctas +=1
        else:
            print("Client sent incorrect answer.")
            incorrecto = ("Incorrecto! La respuesta correcta era: " + correct_answ)
            socket.sendto(pickle.dumps(incorrecto), server_adress)
            
fin = "El juego ha terminado, respondiste correctamente a " + str(correctas) + " preguntas."
print("Game Over, correct answers ", correctas)
socket.sendto(pickle.dumps(fin), server_adress)

- Waiting for a client...
- Listening in 127.0.0.1:6700
Message from client 1: ASK
Server sending question:  The board game Monopoly takes its street names from which real American city?
All answers:  ['Las Vegas, Nevada', 'Duluth, Minnesota', 'Charleston, South Carolina', 'Atlantic City, New Jersey']
Message from client 1: ANSWER
Client answer  Las Vegas, Nevada
Client sent incorrect answer.
Message from client 1: ASK
Server sending question:  In Splatoon, the Squid Sisters are named Tako and Yaki.
All answers:  ['True', 'False']
Message from client 1: ANSWER
Client answer  True
Client sent incorrect answer.
Message from client 1: ASK
Server sending question:  What is the name of the world that the MMO &quot;RuneScape&quot; takes place in?
All answers:  ['Glindor', 'Azeroth', 'Zaros', 'Gielinor']
Message from client 1: ANSWER
Client answer  Zaros
Client sent incorrect answer.
Message from client 1: ASK
Server sending question:  In the 2016 Global Peace Index poll, out of 163 countries

78